In [1]:
import pandas  as  pd
import numpy as  np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import random

# Edad

In [2]:
edad_genero=pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/entrega6/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')]
                    )

In [3]:
edad_genero.drop_duplicates('idpostulante',keep='last',inplace=True)
edad_genero.rename(columns = {'fechanacimiento':'edad'}, inplace = True)
fechaHoy = pd.Timestamp(DT.datetime.now())
edad_genero['edad'] = pd.to_datetime(edad_genero['edad'],errors = 'coerce')
edad_genero['edad'] = (fechaHoy - edad_genero['edad']).astype('<m8[Y]')
edad_genero.head()

,idpostulante,edad,sexo
4,eo2p,37.0,MASC
8,1d2B,42.0,MASC
14,EBO0,44.0,FEM
32,a6MKW,44.0,MASC
36,6MWd4,43.0,MASC


In [4]:
edad_genero = edad_genero.reset_index()
edad_genero.drop(columns=['index'],inplace=True)

In [8]:
no_declara=edad_genero[(edad_genero['sexo']=='NO_DECLARA')]
sexo=['FEM','MASC']
sexo_al_azar=np.random.choice(sexo,25342,p=[0.6,0.4])
azar=pd.DataFrame(sexo_al_azar)
azar.rename(columns={0:'sexo_azar'},inplace=True)

In [9]:
no_declara_sinIndex=no_declara.reset_index()
no_declara_sinIndex.drop(columns=['index'],inplace=True)

In [10]:
sexo_alAzar=no_declara_sinIndex.join(azar)
sexo_alAzar['sexo_azar'].value_counts()

FEM     15180
MASC    10162
Name: sexo_azar, dtype: int64

In [11]:
edad_genero_sinDeclara=edad_genero.merge(sexo_alAzar,on='idpostulante',how='left')
edad_genero_sinDeclara.head()

,idpostulante,edad_x,sexo_x,edad_y,sexo_y,sexo_azar
0,eo2p,37.0,MASC,NaN,NaN,NaN
1,1d2B,42.0,MASC,NaN,NaN,NaN
2,EBO0,44.0,FEM,NaN,NaN,NaN
3,a6MKW,44.0,MASC,NaN,NaN,NaN
4,6MWd4,43.0,MASC,NaN,NaN,NaN


In [13]:
edad_genero_sinDeclara.drop(columns=['edad_y','sexo_y'],inplace=True)
edad_genero_sinDeclara['sexo_x'].value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo_x, dtype: int64

In [14]:
def  sexoABool (sexo):
    if (sexo == 'MASC'):
        return 1
    if (sexo == 'FEM'):
        return 0

In [15]:
edad_genero_sinDeclara['sexo']=edad_genero_sinDeclara['sexo_x'].apply(sexoABool)
edad_genero_sinDeclara['sexo_nuevo_azar']=edad_genero_sinDeclara['sexo_azar'].apply(sexoABool)
edad_genero_sinDeclara['sexo_nuevo_azar'].fillna(0,inplace=True)
edad_genero_sinDeclara.drop(columns=['sexo_x','sexo_azar'],inplace=True)
edad_genero_sinDeclara.head()

,idpostulante,edad_x,sexo,sexo_nuevo_azar
0,eo2p,37.0,1.0,0.0
1,1d2B,42.0,1.0,0.0
2,EBO0,44.0,0.0,0.0
3,a6MKW,44.0,1.0,0.0
4,6MWd4,43.0,1.0,0.0


In [16]:
edad_genero_sinDeclara['genero_definitivo']=edad_genero_sinDeclara['sexo'] + edad_genero_sinDeclara['sexo_nuevo_azar']
edad_genero_sinDeclara.head()

,idpostulante,edad_x,sexo,sexo_nuevo_azar,genero_definitivo
0,eo2p,37.0,1.0,0.0,1.0
1,1d2B,42.0,1.0,0.0,1.0
2,EBO0,44.0,0.0,0.0,0.0
3,a6MKW,44.0,1.0,0.0,1.0
4,6MWd4,43.0,1.0,0.0,1.0


In [17]:
edad_genero_sinDeclara['genero_definitivo']=edad_genero_sinDeclara['genero_definitivo'].astype(bool)
edad_genero_sinDeclara.head()

,idpostulante,edad_x,sexo,sexo_nuevo_azar,genero_definitivo
0,eo2p,37.0,1.0,0.0,True
1,1d2B,42.0,1.0,0.0,True
2,EBO0,44.0,0.0,0.0,False
3,a6MKW,44.0,1.0,0.0,True
4,6MWd4,43.0,1.0,0.0,True


In [18]:
edad_genero_sinDeclara.drop(columns=['sexo','sexo_nuevo_azar'],inplace=True)
edad_genero_sinDeclara.rename(columns={'edad_x':'edad','genero_definitivo':'sexo_masculino'},inplace=True)
edad_genero_sinDeclara.head()

,idpostulante,edad,sexo_masculino
0,eo2p,37.0,True
1,1d2B,42.0,True
2,EBO0,44.0,False
3,a6MKW,44.0,True
4,6MWd4,43.0,True


In [19]:
media=edad_genero_sinDeclara['edad'].mean()
edad_genero_sinDeclara['edad'].fillna(int(random.expovariate(1/media)),inplace = True)

In [20]:
def recalcularEdad(edad):
    if ((edad < 18) or (edad > 66)):
        nueva_edad = int(random.expovariate(1/media))
        while ((nueva_edad < 18) or (nueva_edad>66)):
            nueva_edad = int(random.expovariate(1/media))
        return nueva_edad
    return edad

In [21]:
edad_genero_sinDeclara['edad']=edad_genero_sinDeclara['edad'].apply(recalcularEdad)

In [22]:
edad_genero_sinDeclara['edad'].value_counts().sort_index()

18.0     7778
19.0    14509
20.0    18794
21.0    22196
22.0    24087
23.0    25996
24.0    26256
25.0    26750
26.0    27169
27.0    26747
28.0    23883
29.0    22761
30.0    20782
31.0    19794
32.0    17958
33.0    15800
34.0    14805
35.0    13839
36.0    13142
37.0    12255
38.0    11337
39.0    10731
40.0     9544
41.0     8566
42.0     7826
43.0     7103
44.0     6066
45.0     5341
46.0     5005
47.0     4551
48.0     4090
49.0     3518
50.0     3196
51.0     2852
52.0     2600
53.0     2383
54.0     2126
55.0     1945
56.0     1857
57.0     1672
58.0     1402
59.0     1157
60.0      942
61.0      840
62.0      700
63.0      543
64.0      449
65.0      383
66.0      381
Name: edad, dtype: int64

# Educacion

In [23]:
educacion = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/entrega6/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')]
                      )

In [24]:
def estudioSuperior(nivel):
    if ((nivel=='Posgrado') or (nivel =='Doctorado') or (nivel=='Master')):
        return 1
    else:
        return 0

In [25]:
def estudioTerciarioOtro(nivel):
    if ((nivel=='Otro') or (nivel =='Terciario/Técnico')):
        return 1
    else:
        return 0

In [26]:
graduados=(educacion['estado'].astype(str)== 'Graduado').astype(int)
en_curso=(educacion['estado'].astype(str)== 'En Curso').astype(int)
universitarios=(educacion['nombre'].astype(str) == 'Universitario').astype(int)
secundario=(educacion['nombre'].astype(str) == 'Secundario').astype(int)
mayor_a_universitario=educacion['nombre'].apply(estudioSuperior)
terciario_otro=educacion['nombre'].apply(estudioTerciarioOtro)

In [27]:
educacion['titulo_universitario_graduado']=universitarios*graduados
educacion['titulo_secundario_graduado']=secundario*graduados
educacion['titulo_superior_graduado']=mayor_a_universitario*graduados
educacion['titulo_terciario/otro_graduado']=terciario_otro*graduados
educacion['titulo_universitario_cursando']=universitarios*en_curso
educacion['titulo_secundario_cursando']=secundario*en_curso
educacion['titulo_superior_cursando']=mayor_a_universitario*en_curso
educacion['titulo_terciario/otro_cursando']=terciario_otro*en_curso

In [28]:
titulos = educacion.groupby('idpostulante').agg({'titulo_universitario_graduado':'max',
                                                  'titulo_secundario_graduado':'max',
                                                  'titulo_superior_graduado':'max',
                                                  'titulo_terciario/otro_graduado':'max',
                                                  'titulo_universitario_cursando':'max',
                                                  'titulo_secundario_cursando':'max',
                                                  'titulo_superior_cursando':'max',
                                                  'titulo_terciario/otro_cursando':'max'})
titulos.reset_index(inplace = True)

titulos.head()

,idpostulante,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,titulo_terciario/otro_cursando
0,0z5Dmrd,1,0,0,0,0,0,0,0
1,0z5JW1r,0,1,0,1,0,0,0,0
2,0z5VvGv,0,1,0,0,0,0,0,0
3,0zB01pE,0,0,0,1,0,0,0,0
4,0zB026d,0,1,0,1,0,0,0,0


In [29]:
usuarios = edad_genero_sinDeclara.merge(titulos, on = 'idpostulante', how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,titulo_terciario/otro_cursando
0,eo2p,37.0,True,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1d2B,42.0,True,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EBO0,44.0,False,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,a6MKW,44.0,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,6MWd4,43.0,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
usuarios['titulo_universitario_graduado'] = usuarios['titulo_universitario_graduado'].astype(bool)
usuarios['titulo_secundario_graduado'] = usuarios['titulo_secundario_graduado'].astype(bool)
usuarios['titulo_superior_graduado'] = usuarios['titulo_superior_graduado'].astype(bool)
usuarios['titulo_terciario/otro_graduado'] = usuarios['titulo_terciario/otro_graduado'].astype(bool)
usuarios['titulo_universitario_cursando'] = usuarios['titulo_universitario_cursando'].astype(bool)
usuarios['titulo_secundario_cursando'] = usuarios['titulo_secundario_cursando'].astype(bool)
usuarios['titulo_superior_cursando'] = usuarios['titulo_superior_cursando'].astype(bool)
usuarios['titulo_terciario/otro_cursando'] = usuarios['titulo_terciario/otro_cursando'].astype(bool)
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,titulo_terciario/otro_cursando
0,eo2p,37.0,True,True,True,True,False,False,False,False,False
1,1d2B,42.0,True,False,False,False,False,True,False,False,False
2,EBO0,44.0,False,True,False,True,False,False,False,False,False
3,a6MKW,44.0,True,True,False,True,False,False,False,False,False
4,6MWd4,43.0,True,True,False,False,False,False,False,False,False


# Avisos

In [31]:
avisos=pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/entrega6/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')])
avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [32]:
avisos.drop_duplicates('idaviso', keep = 'last', inplace = True)

In [33]:
avisos.drop(columns=['mapacalle','ciudad','idpais'],inplace = True)

In [34]:
avisos['Full-time'] = (avisos['tipo_de_trabajo'].astype(str) == 'Full-time').astype(bool)
avisos['Part-time'] = (avisos['tipo_de_trabajo'].astype(str) == 'Part-time').astype(bool)
avisos['Pasantia'] = (avisos['tipo_de_trabajo'].astype(str) == 'Pasantia').astype(bool)
avisos['Por Horas'] = (avisos['tipo_de_trabajo'].astype(str) == 'Por Horas').astype(bool)

In [35]:
avisos.rename(columns = {'nombre_zona':'GBA'},inplace = True)
avisos['GBA'] = (avisos['GBA'].astype(str) == 'Gran Buenos Aires').astype(bool)
avisos.head()

,idaviso,titulo,descripcion,GBA,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Full-time,Part-time,Pasantia,Por Horas
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,True,False,False,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,True,False,False,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,Full-time,Junior,Administración,Unión Personal,True,False,False,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,Full-time,Junior,Distribución,Asoko Tempo SA,True,False,False,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,Full-time,Senior / Semi-Senior,Comercial,JELS SRL,True,False,False,False


In [36]:
avisos.drop(columns = 'tipo_de_trabajo', inplace = True)
avisos.rename(columns = {'nivel_laboral':'nivel'}, inplace = True)
avisos['senior'] = (avisos['nivel'].astype(str) == 'Senior / Semi-Senior').astype(bool)
avisos['junior'] = (avisos['nivel'].astype(str) == 'Junior').astype(bool)
avisos['otro'] = (avisos['nivel'].astype(str) == 'Otro').astype(bool)
avisos['Jefe'] = (avisos['nivel'].astype(str) == 'Jefe / Supervisor / Responsable').astype(bool)
avisos['Gerencia'] = (avisos['nivel'].astype(str) == 'Gerencia')

In [37]:
top_areas=['Ventas','Administración','Producción','Comercial','Atención al Cliente','Recepcionista','Call Center','Telemarketing','Tesorería','Mantenimiento y Limpieza']
for area in top_areas:
    avisos[area] = (avisos['nombre_area'] == area).astype(bool)
    
avisos.head()

,idaviso,titulo,descripcion,GBA,nivel,nombre_area,denominacion_empresa,Full-time,Part-time,Pasantia,...,Ventas,Administración,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,True,False,False,...,False,False,True,False,False,False,False,False,False,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,True,False,False,...,False,False,False,True,False,False,False,False,False,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,Junior,Administración,Unión Personal,True,False,False,...,False,True,False,False,False,False,False,False,False,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,Junior,Distribución,Asoko Tempo SA,True,False,False,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,Senior / Semi-Senior,Comercial,JELS SRL,True,False,False,...,False,False,False,True,False,False,False,False,False,False


In [38]:
avisos.drop(columns = ['titulo','descripcion','denominacion_empresa','nivel'],inplace = True)

In [39]:
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,Pasantia,Por Horas,senior,junior,otro,...,Ventas,Administración,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza
4,1000872556,True,Producción,True,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
8,9240880,False,Comercial,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
19,1110185164,True,Administración,True,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
21,1110513885,True,Distribución,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,True,Comercial,True,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


# Vistas

In [40]:
Vistas = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv'),
                          pd.read_csv('data/entrega6/fiuba_3_vistas.csv'),
                          pd.read_csv('data/Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

In [41]:
Vistas['visitas']=1
Vistas.rename(columns={'idAviso':'idaviso'}, inplace = True)

In [42]:
Vistas=Vistas.groupby(['idpostulante','idaviso'])\
                           .agg({'visitas':'sum'})\
                           .reset_index()

In [43]:
Vistas.head()

,idpostulante,idaviso,visitas
0,0002q,1789742,4
1,0002q,1807692,6
2,0002q,1808090,5
3,0005E,1744005,3
4,0005E,1785430,2


# Postulaciones

In [44]:
postulaciones = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv'),
                           pd.read_csv('data/entrega6/fiuba_4_postulaciones.csv')])

In [45]:
postulaciones.drop(columns = 'fechapostulacion', inplace = True)
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'],keep = 'first',inplace = True)

In [46]:
postulaciones=postulaciones.merge(avisos[['idaviso','nombre_area']],
                                    on  = 'idaviso',
                                    how = 'left')

In [47]:
areas_por_postulante = postulaciones.groupby('idpostulante')\
                .agg({'nombre_area': lambda x: list(x),
                      'idaviso':'count'})

areas_por_postulante.head()

,nombre_area,idaviso
idpostulante,,
0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2
0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7
0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50
0zB01pE,"[Administracion de Seguros, Administracion de ...",3
0zB026d,"[Producción, Producción, Producción, Producció...",10


In [48]:
areas_por_postulante.reset_index(inplace = True)
areas_por_postulante.rename(columns = {'nombre_area':'areas', 'idaviso':'cantidad_postulaciones'},
                            inplace = True)

areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3
4,0zB026d,"[Producción, Producción, Producción, Producció...",10


In [49]:
for area in top_areas:
    print(area)
    lista_area = []
    for i in range(0,len(areas_por_postulante)):
        lista_areas_usuario = areas_por_postulante.loc[i,'areas']
        lista_area.append(lista_areas_usuario.count(area))
    
    areas_por_postulante['postulaciones_'+ area] = pd.Series(lista_area)

Ventas
Administración
Producción
Comercial
Atención al Cliente
Recepcionista
Call Center
Telemarketing
Tesorería
Mantenimiento y Limpieza


In [50]:
areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Administración,postulaciones_Producción,postulaciones_Comercial,postulaciones_Atención al Cliente,postulaciones_Recepcionista,postulaciones_Call Center,postulaciones_Telemarketing,postulaciones_Tesorería,postulaciones_Mantenimiento y Limpieza
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2,0,0,0,0,0,0,0,0,0,0
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7,1,0,0,1,0,0,0,0,0,0
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50,13,1,0,7,11,0,4,2,1,0
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3,0,0,0,0,0,0,0,0,0,0
4,0zB026d,"[Producción, Producción, Producción, Producció...",10,1,0,8,0,0,0,0,0,0,0


In [51]:
usuarios=usuarios.merge(areas_por_postulante.drop(columns = 'areas'),
                          on = 'idpostulante',
                          how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,...,postulaciones_Ventas,postulaciones_Administración,postulaciones_Producción,postulaciones_Comercial,postulaciones_Atención al Cliente,postulaciones_Recepcionista,postulaciones_Call Center,postulaciones_Telemarketing,postulaciones_Tesorería,postulaciones_Mantenimiento y Limpieza
0,eo2p,37.0,True,True,True,True,False,False,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1d2B,42.0,True,False,False,False,False,True,False,False,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EBO0,44.0,False,True,False,True,False,False,False,False,...,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0
3,a6MKW,44.0,True,True,False,True,False,False,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6MWd4,43.0,True,True,False,False,False,False,False,False,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
usuarios.isnull().sum()

idpostulante                                   0
edad                                           0
sexo_masculino                                 0
titulo_universitario_graduado                  0
titulo_secundario_graduado                     0
titulo_superior_graduado                       0
titulo_terciario/otro_graduado                 0
titulo_universitario_cursando                  0
titulo_secundario_cursando                     0
titulo_superior_cursando                       0
titulo_terciario/otro_cursando                 0
cantidad_postulaciones                    155524
postulaciones_Ventas                      155524
postulaciones_Administración              155524
postulaciones_Producción                  155524
postulaciones_Comercial                   155524
postulaciones_Atención al Cliente         155524
postulaciones_Recepcionista               155524
postulaciones_Call Center                 155524
postulaciones_Telemarketing               155524
postulaciones_Tesore

In [53]:
usuarios.fillna(0, inplace = True)
usuarios.isnull().sum()

idpostulante                              0
edad                                      0
sexo_masculino                            0
titulo_universitario_graduado             0
titulo_secundario_graduado                0
titulo_superior_graduado                  0
titulo_terciario/otro_graduado            0
titulo_universitario_cursando             0
titulo_secundario_cursando                0
titulo_superior_cursando                  0
titulo_terciario/otro_cursando            0
cantidad_postulaciones                    0
postulaciones_Ventas                      0
postulaciones_Administración              0
postulaciones_Producción                  0
postulaciones_Comercial                   0
postulaciones_Atención al Cliente         0
postulaciones_Recepcionista               0
postulaciones_Call Center                 0
postulaciones_Telemarketing               0
postulaciones_Tesorería                   0
postulaciones_Mantenimiento y Limpieza    0
dtype: int64

## Postulaciones por aviso

In [54]:
postulaciones_por_aviso=postulaciones.groupby('idaviso')\
                                        .agg({'idpostulante':'count'})

In [55]:
postulaciones_por_aviso.reset_index(inplace = True)
postulaciones_por_aviso.rename(columns = {'idpostulante':'postulaciones_aviso'}, inplace = True)
postulaciones_por_aviso.head()

,idaviso,postulaciones_aviso
0,8725750,137
1,11740890,13
2,12543760,1080
3,12812680,159
4,17903700,175


In [56]:
avisos = avisos.merge(postulaciones_por_aviso, on = 'idaviso', how = 'left')
avisos['postulaciones_aviso'].fillna(0,inplace = True)
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,Pasantia,Por Horas,senior,junior,otro,...,Administración,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso
0,1000872556,True,Producción,True,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,0.0
1,9240880,False,Comercial,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,0.0
2,1110185164,True,Administración,True,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,0.0
3,1110513885,True,Distribución,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,0.0
4,1111034024,True,Comercial,True,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,0.0


In [57]:
postulaciones.drop(columns = 'nombre_area', inplace = True)
postulaciones = postulaciones.merge(avisos, 
                                    on = 'idaviso',
                                    how = 'left')
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,Pasantia,Por Horas,senior,junior,...,Administración,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,2391.0
1,1111920714,NM5M,True,Telemarketing,False,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,524.0
2,1112346945,NM5M,True,Telemarketing,True,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,1126.0
3,1112345547,NM5M,True,Telemarketing,True,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,1804.0
4,1112237522,5awk,True,Contabilidad,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,193.0


In [58]:
postulaciones['postulacion']=True
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 25 columns):
idaviso                     int64
idpostulante                object
GBA                         object
nombre_area                 object
Full-time                   object
Part-time                   object
Pasantia                    object
Por Horas                   object
senior                      object
junior                      object
otro                        object
Jefe                        object
Gerencia                    object
Ventas                      object
Administración              object
Producción                  object
Comercial                   object
Atención al Cliente         object
Recepcionista               object
Call Center                 object
Telemarketing               object
Tesorería                   object
Mantenimiento y Limpieza    object
postulaciones_aviso         float64
postulacion                 bool
dtypes: bool(1

In [59]:
postulaciones = postulaciones.merge(usuarios, on = 'idpostulante', how = 'left')
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,Pasantia,Por Horas,senior,junior,...,postulaciones_Ventas,postulaciones_Administración,postulaciones_Producción,postulaciones_Comercial,postulaciones_Atención al Cliente,postulaciones_Recepcionista,postulaciones_Call Center,postulaciones_Telemarketing,postulaciones_Tesorería,postulaciones_Mantenimiento y Limpieza
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0
1,1111920714,NM5M,True,Telemarketing,False,True,False,False,True,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0
2,1112346945,NM5M,True,Telemarketing,True,False,False,False,True,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0
3,1112345547,NM5M,True,Telemarketing,True,False,False,False,True,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0
4,1112237522,5awk,True,Contabilidad,True,False,False,False,True,False,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 46 columns):
idaviso                                   int64
idpostulante                              object
GBA                                       object
nombre_area                               object
Full-time                                 object
Part-time                                 object
Pasantia                                  object
Por Horas                                 object
senior                                    object
junior                                    object
otro                                      object
Jefe                                      object
Gerencia                                  object
Ventas                                    object
Administración                            object
Producción                                object
Comercial                                 object
Atención al Cliente                       object
Recepc

In [65]:
columnas_booleanas = ['GBA','Full-time','Part-time', 'senior', 'junior', 'otro', 'Jefe', 'Gerencia','Ventas',
                      'Administración','Producción','Comercial','Atención al Cliente','Recepcionista','Call Center',
                      'Telemarketing','Tesorería','Mantenimiento y Limpieza','Pasantia','Por Horas']

In [66]:
for columna in columnas_booleanas:
    postulaciones[columna] = postulaciones[columna].astype(bool)
    
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 46 columns):
idaviso                                   int64
idpostulante                              object
GBA                                       bool
nombre_area                               object
Full-time                                 bool
Part-time                                 bool
Pasantia                                  bool
Por Horas                                 bool
senior                                    bool
junior                                    bool
otro                                      bool
Jefe                                      bool
Gerencia                                  bool
Ventas                                    bool
Administración                            bool
Producción                                bool
Comercial                                 bool
Atención al Cliente                       bool
Recepcionista                       

In [68]:
postulaciones  = postulaciones.merge(Vistas,
                                    on = ['idaviso','idpostulante'],
                                    how = 'left')
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 47 columns):
idaviso                                   int64
idpostulante                              object
GBA                                       bool
nombre_area                               object
Full-time                                 bool
Part-time                                 bool
Pasantia                                  bool
Por Horas                                 bool
senior                                    bool
junior                                    bool
otro                                      bool
Jefe                                      bool
Gerencia                                  bool
Ventas                                    bool
Administración                            bool
Producción                                bool
Comercial                                 bool
Atención al Cliente                       bool
Recepcionista                       

In [69]:
postulaciones['visitas'].fillna(0,inplace = True)
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,Pasantia,Por Horas,senior,junior,...,postulaciones_Administración,postulaciones_Producción,postulaciones_Comercial,postulaciones_Atención al Cliente,postulaciones_Recepcionista,postulaciones_Call Center,postulaciones_Telemarketing,postulaciones_Tesorería,postulaciones_Mantenimiento y Limpieza,visitas
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
1,1111920714,NM5M,True,Telemarketing,False,True,False,False,True,False,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
2,1112346945,NM5M,True,Telemarketing,True,False,False,False,True,False,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
3,1112345547,NM5M,True,Telemarketing,True,False,False,False,True,False,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0
4,1112237522,5awk,True,Contabilidad,True,False,False,False,True,False,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
postulaciones.isnull().sum()

idaviso                                        0
idpostulante                                   0
GBA                                            0
nombre_area                               415823
Full-time                                      0
Part-time                                      0
Pasantia                                       0
Por Horas                                      0
senior                                         0
junior                                         0
otro                                           0
Jefe                                           0
Gerencia                                       0
Ventas                                         0
Administración                                 0
Producción                                     0
Comercial                                      0
Atención al Cliente                            0
Recepcionista                                  0
Call Center                                    0
Telemarketing       

# Generando No Postulaciones

In [71]:
no_postulaciones = usuarios.sample(10000)
for i in range(1,300):
    no_postulaciones = pd.concat([no_postulaciones, usuarios.sample(25000)])

In [72]:
avisos_no_postulaciones = avisos.sample(10000)
for i in range(1,300):
    avisos_no_postulaciones = pd.concat([avisos_no_postulaciones,avisos.sample(25000)])

In [73]:
no_postulaciones.reset_index(inplace = True)
no_postulaciones.drop(columns = 'index', inplace = True)
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,...,postulaciones_Ventas,postulaciones_Administración,postulaciones_Producción,postulaciones_Comercial,postulaciones_Atención al Cliente,postulaciones_Recepcionista,postulaciones_Call Center,postulaciones_Telemarketing,postulaciones_Tesorería,postulaciones_Mantenimiento y Limpieza
0,lDL6vN1,31.0,True,True,True,True,True,True,True,True,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,0.0,2.0,0.0
1,MV6RwRN,36.0,True,False,True,False,False,True,False,False,...,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,PmqVM2Y,27.0,False,True,False,False,True,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,jkj2Z4Y,41.0,False,False,False,False,False,True,False,False,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,vVjRoA1,29.0,False,False,True,False,False,False,False,False,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0


In [74]:
avisos_no_postulaciones.reset_index(inplace = True)
avisos_no_postulaciones.drop(columns = 'index', inplace = True)
avisos_no_postulaciones.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,Pasantia,Por Horas,senior,junior,otro,...,Administración,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso
0,1112295765,True,Mantenimiento,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,188.0
1,1111883545,True,Servicios,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,30.0
2,1112425495,False,Comercial,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,151.0
3,1112113451,True,Producción,True,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,0.0
4,1112487192,True,Ventas Internacionales/Exportación,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,0.0


In [75]:
no_postulaciones = no_postulaciones.join(avisos_no_postulaciones)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7485000 entries, 0 to 7484999
Data columns (total 45 columns):
idpostulante                              object
edad                                      float64
sexo_masculino                            bool
titulo_universitario_graduado             bool
titulo_secundario_graduado                bool
titulo_superior_graduado                  bool
titulo_terciario/otro_graduado            bool
titulo_universitario_cursando             bool
titulo_secundario_cursando                bool
titulo_superior_cursando                  bool
titulo_terciario/otro_cursando            bool
cantidad_postulaciones                    float64
postulaciones_Ventas                      float64
postulaciones_Administración              float64
postulaciones_Producción                  float64
postulaciones_Comercial                   float64
postulaciones_Atención al Cliente         float64
postulaciones_Recepcionista               float64
postulaciones_C

In [76]:
no_postulaciones['postulacion'] = False
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,...,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso,postulacion
0,lDL6vN1,31.0,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,188.0,False
1,MV6RwRN,36.0,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,30.0,False
2,PmqVM2Y,27.0,False,True,False,False,True,False,False,True,...,False,True,False,False,False,False,False,False,151.0,False
3,jkj2Z4Y,41.0,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,False,False,0.0,False
4,vVjRoA1,29.0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.0,False


In [77]:
no_postulaciones = no_postulaciones.merge(Vistas,
                                          on = ['idaviso','idpostulante'],
                                          how = 'left')
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,titulo_secundario_cursando,titulo_superior_cursando,...,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso,postulacion,visitas
0,lDL6vN1,31.0,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,188.0,False,NaN
1,MV6RwRN,36.0,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,30.0,False,NaN
2,PmqVM2Y,27.0,False,True,False,False,True,False,False,True,...,True,False,False,False,False,False,False,151.0,False,NaN
3,jkj2Z4Y,41.0,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,0.0,False,NaN
4,vVjRoA1,29.0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,0.0,False,NaN


In [78]:
no_postulaciones['visitas'].fillna(0, inplace = True)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7485000 entries, 0 to 7484999
Data columns (total 47 columns):
idpostulante                              object
edad                                      float64
sexo_masculino                            bool
titulo_universitario_graduado             bool
titulo_secundario_graduado                bool
titulo_superior_graduado                  bool
titulo_terciario/otro_graduado            bool
titulo_universitario_cursando             bool
titulo_secundario_cursando                bool
titulo_superior_cursando                  bool
titulo_terciario/otro_cursando            bool
cantidad_postulaciones                    float64
postulaciones_Ventas                      float64
postulaciones_Administración              float64
postulaciones_Producción                  float64
postulaciones_Comercial                   float64
postulaciones_Atención al Cliente         float64
postulaciones_Recepcionista               float64
postulaciones_C

In [79]:
no_postulaciones = no_postulaciones[postulaciones.columns]
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7485000 entries, 0 to 7484999
Data columns (total 47 columns):
idaviso                                   int64
idpostulante                              object
GBA                                       bool
nombre_area                               object
Full-time                                 bool
Part-time                                 bool
Pasantia                                  bool
Por Horas                                 bool
senior                                    bool
junior                                    bool
otro                                      bool
Jefe                                      bool
Gerencia                                  bool
Ventas                                    bool
Administración                            bool
Producción                                bool
Comercial                                 bool
Atención al Cliente                       bool
Recepcionista                       

In [80]:
postulaciones = pd.concat([postulaciones,no_postulaciones])

In [81]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'], keep = 'first',inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14083028 entries, 0 to 7484999
Data columns (total 47 columns):
idaviso                                   int64
idpostulante                              object
GBA                                       bool
nombre_area                               object
Full-time                                 bool
Part-time                                 bool
Pasantia                                  bool
Por Horas                                 bool
senior                                    bool
junior                                    bool
otro                                      bool
Jefe                                      bool
Gerencia                                  bool
Ventas                                    bool
Administración                            bool
Producción                                bool
Comercial                                 bool
Atención al Cliente                       bool
Recepcionista                      

In [82]:
postulaciones['postulaciones_aviso'].fillna(0, inplace = True)
postulaciones.isnull().sum()

idaviso                                        0
idpostulante                                   0
GBA                                            0
nombre_area                               415823
Full-time                                      0
Part-time                                      0
Pasantia                                       0
Por Horas                                      0
senior                                         0
junior                                         0
otro                                           0
Jefe                                           0
Gerencia                                       0
Ventas                                         0
Administración                                 0
Producción                                     0
Comercial                                      0
Atención al Cliente                            0
Recepcionista                                  0
Call Center                                    0
Telemarketing       

# Generando el set Final

In [83]:
test_final= pd.read_csv('data/test_final_100k.csv')

In [84]:
test_final = test_final.merge(usuarios, on='idpostulante', how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,...,postulaciones_Ventas,postulaciones_Administración,postulaciones_Producción,postulaciones_Comercial,postulaciones_Atención al Cliente,postulaciones_Recepcionista,postulaciones_Call Center,postulaciones_Telemarketing,postulaciones_Tesorería,postulaciones_Mantenimiento y Limpieza
0,0,739260,6M9ZQR,42.0,False,True,False,True,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,739260,6v1xdL,30.0,True,False,False,False,True,False,...,4.0,0.0,2.0,5.0,14.0,1.0,4.0,0.0,0.0,1.0
2,2,739260,ezRKm9,36.0,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,758580,1Q35ej,58.0,True,True,False,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [85]:
test_final = test_final.merge(avisos,
                            on = 'idaviso',
                            how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,...,Administración,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso
0,0,739260,6M9ZQR,42.0,False,True,False,True,False,False,...,False,False,True,False,False,False,False,False,False,0.0
1,1,739260,6v1xdL,30.0,True,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,0.0
2,2,739260,ezRKm9,36.0,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,0.0
3,3,758580,1Q35ej,58.0,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,0.0
4,4,758580,EAN4J6,32.0,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,0.0


In [86]:
test_final = test_final.merge(Vistas, on = ['idaviso','idpostulante'], how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario_graduado,titulo_secundario_graduado,titulo_superior_graduado,titulo_terciario/otro_graduado,titulo_universitario_cursando,...,Producción,Comercial,Atención al Cliente,Recepcionista,Call Center,Telemarketing,Tesorería,Mantenimiento y Limpieza,postulaciones_aviso,visitas
0,0,739260,6M9ZQR,42.0,False,True,False,True,False,False,...,False,True,False,False,False,False,False,False,0.0,NaN
1,1,739260,6v1xdL,30.0,True,False,False,False,True,False,...,False,True,False,False,False,False,False,False,0.0,NaN
2,2,739260,ezRKm9,36.0,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,0.0,NaN
3,3,758580,1Q35ej,58.0,True,True,False,True,False,False,...,False,False,False,False,False,False,False,False,0.0,NaN
4,4,758580,EAN4J6,32.0,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,0.0,NaN


In [87]:
test_final['visitas'].fillna(0, inplace = True)
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 47 columns):
id                                        100000 non-null int64
idaviso                                   100000 non-null int64
idpostulante                              100000 non-null object
edad                                      100000 non-null float64
sexo_masculino                            100000 non-null bool
titulo_universitario_graduado             100000 non-null bool
titulo_secundario_graduado                100000 non-null bool
titulo_superior_graduado                  100000 non-null bool
titulo_terciario/otro_graduado            100000 non-null bool
titulo_universitario_cursando             100000 non-null bool
titulo_secundario_cursando                100000 non-null bool
titulo_superior_cursando                  100000 non-null bool
titulo_terciario/otro_cursando            100000 non-null bool
cantidad_postulaciones                    100000 non-null float6

# Empieza Random Forest

In [88]:
datos = postulaciones.drop(columns = ['idpostulante','idaviso','postulacion','nombre_area'])
categorias = postulaciones['postulacion']

In [89]:
test_size = 0.33
random_state = 0

In [90]:
datos_train , datos_test, cat_train, cat_test = train_test_split(datos,
                                                               categorias,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [94]:
clf = RandomForestClassifier(n_jobs=1,random_state=0)

In [95]:
clf.fit(datos_train,cat_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [96]:
cat_pred = clf.predict(datos_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 92.96%


In [97]:
final_test_data = test_final[datos.columns]

In [98]:
prediccion = clf.predict(final_test_data)
prediccion = prediccion.astype(int)

In [99]:
test_final['sepostulo'] = prediccion
entregable = test_final[['id','sepostulo']]

In [100]:
entregable.index = entregable['id']
entregable.drop(columns = 'id', inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [101]:
entregable.to_csv('submit_random05.csv')